In [300]:
# Import libraries
from urllib.request import urljoin
from bs4 import BeautifulSoup
import requests
from urllib.request import urlparse
import re
import pandas as pd
# # Set for storing urls with same domain

# # Method for crawling a url at next level
def level_crawler(input_url,temp_urls,count_publications,df,j):
    data = []
    current_url_domain = urlparse(input_url).netloc
#   # Creates beautiful soup object to extract html tags
    beautiful_soup_object = BeautifulSoup(requests.get(input_url).content, "lxml")
#   # Access all anchor tags from input
#   # url page and divide them into internal
#   # and external categories
    for anchor in beautiful_soup_object.findAll("a"):
        href = anchor.attrs.get("href")
        title = anchor.string
        if(href != "" or href != None):
            href = urljoin(input_url, href)
            href_parsed = urlparse(href)
            href = href_parsed.scheme
            href += "://"
            href += href_parsed.netloc
            href += href_parsed.path
            final_parsed_href = urlparse(href)
            is_valid = bool(final_parsed_href.scheme) and bool(
                final_parsed_href.netloc)
            if is_valid:
                if current_url_domain not in href and href not in links_extern:
                    #print("Extern - {}".format(href))
                    links_extern.add(href)
                if current_url_domain in href and href not in links_intern:
                     #if re.search("^https://pureportal.coventry.ac.uk/en/publications/.*$", href):
                        #re.search("^The.*Spain$", href)
                    if '/publications/' in href[36:50] and href[50:] != '':
                        #print("Intern - {}".format(href))
                        count_publications += 1
                        links_intern.add(href)
                        temp_urls.append(href)
                        data.append(href)
                        #print('publications title:',title)
                        df.loc[j, ['publication_link']]  = href
                        df.loc[j, ['publication_title']] = title
                        df,j= get_author_details(href,df,j)
                        j += 1
    return temp_urls,count_publications,df,j,data
def get_author_details(link,df,j):
    source_code = requests.get(link)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text,"lxml")
    for link in soup.findAll('a',{'class':'link person'}):
        href = link.get('href')
        name = link.string
        source_code1 = requests.get(href)
        plain_text1 = source_code1.text
        soup1 = BeautifulSoup(plain_text1,"lxml")
        if(soup1.find('a',{'class':'link primary'}) != None):
            sefa = soup1.find('a',{'class':'link primary'}).string
        elif(soup1.find('a',{'class':'link school'}) != None):
            sefa = soup1.find('a',{'class':'link school'}).string
        if (sefa == 'School of Economics, Finance and Accounting'):
            df.loc[j, ['author_link']]  = href
            df.loc[j, ['author_name']] = name
    publi = soup.find('tr',{'class':'status'})
    date = publi.text.strip('date')
    #print('date:',date)
    #print(abstract,'abs..........')
    if (soup.find('div',{'class':'textblock'})):
        abstract = soup.find('div',{'class':'textblock'}).string
    else: 
        abstract = 'Null'
    #print(abstract,'abs..........')
    if type(abstract) == 'NoneType':
        abstract = ' '
    df.loc[j, ['publication_date']]  = date
    df.loc[j, ['abstract']] = abstract
    #df.loc[j, ['content']] = abstract + name
    return df,j                       
                    
links_intern = set()
url = 'https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/'
depth = 1
p = 0
count_publications = 0  
# # Set for storing urls with different domain
df = pd.DataFrame(columns = ["publication_link","publication_title","publication_date","author_link","author_name","abstract","content"])
j = 0
links_extern = set()
list_url = []
if(depth == 0):
    print("Intern - {}".format(input_url))
elif(depth == 1):
    while p >= 0:
        if p == 0:
            list = []
            print(url,'url page0')
            list_url,count_publications,df,j,data = level_crawler(url,list,count_publications,df,j)
            p += 1
        else:
            new_url = url+'?page='+str(p)
            print(new_url,'url page')
            list_url,count_publications,df,j,data = level_crawler(new_url,list_url,count_publications,df,j)
            p += 1
            if (data == []):
                p = -1
else:
#   # We have used a BFS approach
#   # considering the structure as
#   # a tree. It uses a queue based
#   # approach to traverse
#   # links upto a particular depth.
    queue = []
    queue.append(input_url)
    for j in range(depth):
        for count in range(len(queue)):
            url = queue.pop(0)
            urls = level_crawler(url)
            for i in urls:
                queue.append(i)
print('end')
print('total no of publications',count_publications)
#print(list_url)

https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/ url page0
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=1 url page
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=2 url page
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=3 url page
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=4 url page
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=5 url page
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=6 url page
https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=7 url page
https://purepo

In [343]:
df['content']=df['abstract']+df['author_name']+df['publication_title']
df

,publication_link,publication_title,publication_date,author_link,author_name,abstract,content
0,https://pureportal.coventry.ac.uk/en/publicati...,A bibliometric review of the Waqf literature,Publication statusPublished - Jun 2022,https://pureportal.coventry.ac.uk/en/persons/r...,Rashedul Hasan,Waqf received research attention due to its ab...,Waqf received research attention due to its ab...
1,https://pureportal.coventry.ac.uk/en/publicati...,A note on COVID-19 instigated maximum drawdown...,Publication statusPublished - May 2022,https://pureportal.coventry.ac.uk/en/persons/r...,Rashedul Hasan,This study uncovers the impact of the COVID-19...,This study uncovers the impact of the COVID-19...
2,https://pureportal.coventry.ac.uk/en/publicati...,Bank stock valuation theories: do they explain...,Publication statusE-pub ahead of print - 1 Mar...,https://pureportal.coventry.ac.uk/en/persons/a...,Alireza Zarei,None,NaN
3,https://pureportal.coventry.ac.uk/en/publicati...,CEO Duality and Firm Performance: A Systematic...,Publication statusE-pub ahead of print - 25 Ma...,https://pureportal.coventry.ac.uk/en/persons/m...,Mei Yu,This paper systematically reviews 314 empirica...,This paper systematically reviews 314 empirica...
4,https://pureportal.coventry.ac.uk/en/publicati...,CEO Financial Experience and Firms’ Earnings M...,Publication statusSubmitted - 7 Mar 2022,https://pureportal.coventry.ac.uk/en/persons/t...,Thai Nguyen,This study investigates the impact of CEOs’ fi...,This study investigates the impact of CEOs’ fi...
...,...,...,...,...,...,...,...
633,https://pureportal.coventry.ac.uk/en/publicati...,Evaluation of clinical interventions: effectiv...,Publication statusPublished - 1998,NaN,NaN,Null,NaN
634,https://pureportal.coventry.ac.uk/en/publicati...,Market Orientation in the UK Higher Education ...,Publication statusPublished - Jul 1998,NaN,NaN,Null,NaN
635,https://pureportal.coventry.ac.uk/en/publicati...,Developing a comprehensive cross-country econo...,Publication statusPublished - 1997,https://pureportal.coventry.ac.uk/en/persons/s...,Sailesh Tanna,This paper describes a framework for setting u...,This paper describes a framework for setting u...
636,https://pureportal.coventry.ac.uk/en/publicati...,Measuring value added in higher education: a p...,Publication statusPublished - Aug 1995,NaN,NaN,Null,NaN


In [344]:
df['author_name'].unique()

array(['Rashedul Hasan', 'Alireza Zarei', 'Mei Yu', 'Thai Nguyen',
       'Ruth Owusu-Mensah', 'Masud Ibrahim', 'John Ayuk Enombu',
       'Ahmad Abras', 'Hafij Ullah', 'Hany Ahmed', 'Boying Xu',
       'Ibrahim Elmghaamez', 'Andrews Owusu', 'Uchenna Tony-Okeke',
       'Muhammad Shahin Miah', 'Isaiah Oino', 'Francis Awuku Darko',
       'Loai Alsaid', 'Mohammad Khaleq Newaz', nan, 'Huston, S.',
       'Ahmed Saleh', 'Mehdi Hosseini', 'Simon Huston', 'Mehul Chhatbar',
       'Tariq Al Montaser', 'Alaa Alhaj Ismail', 'Angelos Synapis',
       'Mehtap Hisarciklilar', 'Dimitris Serenis', 'Daniel Santamaria',
       'Sarkar Kabir', 'Styliani Panetsidou', 'Sandar Win',
       'Salem Alhababsah', 'Piotr Lis', 'Abdurafiu Noah', 'Junyuan Chen',
       'Mahbub Khan', 'Christopher Muganhu', 'Nikhil Sapre',
       'Luda Ruddock', 'Sailesh Tanna', 'Akin Sharimakin',
       'Simon Horsman', 'Olubunmi Ajala', 'Samir Alamad',
       'Daniel Aghanya', 'Alloysius Egbulonu', 'Ullah, H.',
       'Jaliyya

In [345]:
df['author_name'].nunique()

80

In [346]:
d = df['publication_title'].groupby(df['author_name']).size()
print(d)

author_name
Abdurafiu Noah        18
Ahmad Abras            7
Ahmed Saleh            3
Akin Sharimakin        2
Alaa Alhaj Ismail      5
                      ..
Thai Nguyen            4
Uchenna Tony-Okeke     4
Ullah, H.              2
Wei Song               5
Yu Wang                2
Name: publication_title, Length: 80, dtype: int64


In [347]:
import csv
df.to_csv(path_or_buf ="crawlerdata.csv",sep=',')


In [377]:
df = pd.read_csv('crawlerdata.csv')

In [378]:
content = df['content'].astype(str)
print(content)

0      Waqf received research attention due to its ab...
1      This study uncovers the impact of the COVID-19...
2                                                    nan
3      This paper systematically reviews 314 empirica...
4      This study investigates the impact of CEOs’ fi...
                             ...                        
633                                                  nan
634                                                  nan
635    This paper describes a framework for setting u...
636                                                  nan
637                                                  nan
Name: content, Length: 638, dtype: object


In [379]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

sw = stopwords.words('english')
nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

ps = PorterStemmer()
filtered_docs = []
for doc in content:
    tokens = word_tokenize(doc)
    tmp = ""
    for w in tokens:
        if w not in sw:
            tmp += ps.stem(w) + " "
    filtered_docs.append(tmp)

print(filtered_docs)

[nltk_data] Downloading package stopwords to C:\Users\Bharathi
[nltk_data]     Kondaveeti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Bharathi
[nltk_data]     Kondaveeti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['waqf receiv research attent due abil contribut socio-econom develop . while high volum literari evid islam social financ instrument avail , research motiv studi find divers . therefor , conduct bibliometr analysi waqf literatur understand pattern direct research waqf broadli . we collect 319 articl , review waqf extract scopu databas , cover period exceed 100 year 1914 june 2020 . we employ rstudio , vosview , microsoft excel citat analysi , content , network analys . a systemat review recent public complement bibliometr analysi . alongsid reveal relev scientif actor waqf literatur , research waqf malaysia ; conduct studi wide cash waqf . as result , identifi four research theme waqf studi includ ( 1 ) cash waqf endow , ( 2 ) islam account waqf , ( 3 ) waqf islam social financ , ( 4 ) govern waqf endow . thi first studi provid bibliometr review waqf literatur address gap exist research offer direct futur research could benefit early-car islam financ researchers.rashedul hasana biblio

In [380]:
df['fil_content'] = filtered_docs

In [381]:
df

,Unnamed: 0,publication_link,publication_title,publication_date,author_link,author_name,abstract,content,fil_content
0,0,https://pureportal.coventry.ac.uk/en/publicati...,A bibliometric review of the Waqf literature,Publication statusPublished - Jun 2022,https://pureportal.coventry.ac.uk/en/persons/r...,Rashedul Hasan,Waqf received research attention due to its ab...,Waqf received research attention due to its ab...,waqf receiv research attent due abil contribut...
1,1,https://pureportal.coventry.ac.uk/en/publicati...,A note on COVID-19 instigated maximum drawdown...,Publication statusPublished - May 2022,https://pureportal.coventry.ac.uk/en/persons/r...,Rashedul Hasan,This study uncovers the impact of the COVID-19...,This study uncovers the impact of the COVID-19...,thi studi uncov impact covid-19 islam equiti m...
2,2,https://pureportal.coventry.ac.uk/en/publicati...,Bank stock valuation theories: do they explain...,Publication statusE-pub ahead of print - 1 Mar...,https://pureportal.coventry.ac.uk/en/persons/a...,Alireza Zarei,NaN,NaN,nan
3,3,https://pureportal.coventry.ac.uk/en/publicati...,CEO Duality and Firm Performance: A Systematic...,Publication statusE-pub ahead of print - 25 Ma...,https://pureportal.coventry.ac.uk/en/persons/m...,Mei Yu,This paper systematically reviews 314 empirica...,This paper systematically reviews 314 empirica...,thi paper systemat review 314 empir studi exam...
4,4,https://pureportal.coventry.ac.uk/en/publicati...,CEO Financial Experience and Firms’ Earnings M...,Publication statusSubmitted - 7 Mar 2022,https://pureportal.coventry.ac.uk/en/persons/t...,Thai Nguyen,This study investigates the impact of CEOs’ fi...,This study investigates the impact of CEOs’ fi...,thi studi investig impact ceo ’ financi experi...
...,...,...,...,...,...,...,...,...,...
633,633,https://pureportal.coventry.ac.uk/en/publicati...,Evaluation of clinical interventions: effectiv...,Publication statusPublished - 1998,NaN,NaN,Null,NaN,nan
634,634,https://pureportal.coventry.ac.uk/en/publicati...,Market Orientation in the UK Higher Education ...,Publication statusPublished - Jul 1998,NaN,NaN,Null,NaN,nan
635,635,https://pureportal.coventry.ac.uk/en/publicati...,Developing a comprehensive cross-country econo...,Publication statusPublished - 1997,https://pureportal.coventry.ac.uk/en/persons/s...,Sailesh Tanna,This paper describes a framework for setting u...,This paper describes a framework for setting u...,thi paper describ framework set maintain compr...
636,636,https://pureportal.coventry.ac.uk/en/publicati...,Measuring value added in higher education: a p...,Publication statusPublished - Aug 1995,NaN,NaN,Null,NaN,nan


In [382]:
inverted_index = {}

for i, doc in enumerate(df['fil_content']):
    for term in doc.split(" "):
        if term in inverted_index:
            inverted_index[term].add(i)
        else: inverted_index[term] = {i}
inverted_index

{'waqf': {0},
 'receiv': {0, 292, 423, 434, 457, 536},
 'research': {0,
  3,
  9,
  20,
  21,
  24,
  28,
  32,
  34,
  35,
  38,
  43,
  46,
  49,
  55,
  56,
  59,
  60,
  71,
  72,
  81,
  86,
  107,
  117,
  120,
  121,
  129,
  136,
  165,
  170,
  185,
  202,
  203,
  216,
  222,
  235,
  261,
  292,
  295,
  297,
  309,
  311,
  316,
  317,
  318,
  333,
  342,
  359,
  363,
  389,
  401,
  428,
  430,
  434,
  448,
  454,
  456,
  470,
  485,
  502,
  507,
  516,
  548,
  560,
  571,
  587,
  589,
  626,
  635},
 'attent': {0, 117, 197, 277, 316, 434, 495},
 'due': {0,
  3,
  5,
  22,
  47,
  54,
  72,
  185,
  187,
  203,
  217,
  231,
  316,
  318,
  459,
  485,
  491,
  508,
  548,
  560},
 'abil': {0, 5, 6, 28, 43, 54, 79, 202, 231, 276, 434},
 'contribut': {0,
  4,
  16,
  18,
  30,
  32,
  40,
  43,
  45,
  50,
  53,
  55,
  56,
  59,
  67,
  79,
  93,
  94,
  107,
  129,
  178,
  182,
  220,
  229,
  243,
  295,
  302,
  309,
  316,
  317,
  318,
  424,
  441,
  446,
  4

In [443]:
query = input('enter input query:')

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

sw = stopwords.words('english')
nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

ps = PorterStemmer()
filtered_docs = []
tokens = word_tokenize(query)
tmp = ""
for w in tokens:
    if w not in sw:
        tmp += ps.stem(w) + " "
filtered_docs.append(tmp)

#print(filtered_docs)

x = filtered_docs[0].split(" ")

q = []
for i in range(len(x)):
    if len(x[i]) > 0:
        q.append(x[i])
#print(q)      

p = []
for i in range(len(q)):
    if (q[i] in inverted_index):
        a = inverted_index[q[i]]
        l = sorted(a)
        p.append(l)
        print('l = ',q[i],l)
    else:
        print(q[i],'not found')
for i in p:
    print(i)
for i  in range(len(p)):
    print(p[i])
    for j in range(len(p[i])):
        print(p[i][j])
        print(df.loc[p[i][j]])

        

enter input query:bharathi cherukuri
bharathi not found
cherukuri not found


[nltk_data] Downloading package stopwords to C:\Users\Bharathi
[nltk_data]     Kondaveeti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Bharathi
[nltk_data]     Kondaveeti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [1]:
"""COVID-19 instigated maximum drawdown in Islamic markets versus conventional counterparts
Both Islamic and non-Islamic Asian markets signpost a quicker recovery than the rest of the regions, the Middle East & Africa, Europe, and America.
Informal Entrepreneurship in Africa
CEO Duality and Firm Performance: A Systematic Review and Research Agenda
developments in corporate governance research
Historical Institutionalist Perspective
Myanmar government embarked on the privatisation
impact of CEOs’ financial experience on firms earnings management around M&A
Revisiting the evidence of earnings management prior to merger announcements
impact of CEOs’ financial experience on firms earnings
Liquidity and Short-Run Predictability
bibliometric review
Waqf literature"""

'COVID-19 instigated maximum drawdown in Islamic markets versus conventional counterparts\nBoth Islamic and non-Islamic Asian markets signpost a quicker recovery than the rest of the regions, the Middle East & Africa, Europe, and America.\nInformal Entrepreneurship in Africa\nCEO Duality and Firm Performance: A Systematic Review and Research Agenda\ndevelopments in corporate governance research\nHistorical Institutionalist Perspective\nMyanmar government embarked on the privatisation\nimpact of CEOs’ financial experience on firms earnings management around M&A\nRevisiting the evidence of earnings management prior to merger announcements\nimpact of CEOs’ financial experience on firms earnings\nLiquidity and Short-Run Predictability\nbibliometric review\nWaqf literature'